###逻辑回归的贝叶斯因子

#####失败和温度之间真的有关系吗？

对我们上面分析的批评是*假设*了关系遵循一个logistic模型，这样我们暗含地假设了概率会随温度改变。让我们再次看一下数据。（上图）

有可能实际上这个特定的缺陷序列只是偶然发生的吗？毕竟，我也可以得到类似的图表。（下图）这可能会解释温度之间的大幅度重叠。

In [1]:
figsize(12.5, 6)
subplot(211)
plt.scatter(challenger_data[:, 0], challenger_data[:, 1], s=75,
            color="k", alpha=0.5)
plt.yticks([0, 1])
plt.ylabel("Damage Incident?")
plt.title("(Real) Defects of the Space Shuttle O-Rings vs temperature")

subplot(212)
n = challenger_data.shape[0]
plt.scatter(challenger_data[:, 0], stats.bernoulli.rvs(0.6, size=n),
            s=75, color="k", alpha=0.5)
plt.yticks([0, 1])
plt.ylabel("Damage Incident?")
plt.xlabel("Outside temperature (Farhenhit)")
plt.title("(Artificial) Defects of the Space Shuttle O-Rings vs \
temperature")

NameError: name 'figsize' is not defined

引入贝叶斯因子

*贝叶斯因子*是比较两个模型的一种度量。在我们的例子中，一方面我们相信温度在O型环故障的概率中起着重要作用。另一方面，我们认为没有显著联系，而模式只是偶然出现。我们可以通过比较*观测数据的概率，给定模型*的比率来比较这些模型：

$$ \frac{ P( \text{观测数据} | \text{模型}_1 ) }{  P( \text{观测数据} | \text{模型}_2 ) }$$

计算这个并不明显。为简单起见，让我们从后验分布中选择一组参数（这相当于选择特定的模型）。

In [2]:
alpha_hat = alpha_samples[0, 0]  # select the first alpha sample
beta_hat = beta_samples[0, 0]  # select the first beta sample

p_hat = logistic(temperature, beta_hat, alpha_hat)
print "estimates of probability at observed temperature, defects: "
print np.array(zip(p_hat, temperature, D))[:3, :]
print "..."

NameError: name 'alpha_samples' is not defined

为了计算贝叶斯因子中的分子，我们首先**假定一个模型**，在我们的情况下假定`alpha_hat`，`beta_hat`是真实的，并计算我们观察到的缺陷的概率，等于以下乘积：

$$ P(\; \text{Ber}(\; p(t_i \; | \; \hat{\beta}, \hat{\alpha} )\; ) = D_i \; ),\;\; i=1..N $$

例如，使用上述输出，第一个计算，$i=0$，如下所示

\begin{align}
& p = p(t_1 \; | \; \hat{\beta}, \hat{\alpha} ) = 0.667 \\\\
& d = D_0 = 0 \\\\
& \Rightarrow \; P( \; \text{Ber}(p) = d ) = ?? 
\end{align}

这种情况发生的概率为$(1-0.667) = 0.333\; $（回想伯努利随机变量$\text{Ber}(p)$的定义：它以概率$p$等于1，以概率$1-p$等于0）。由于每个观察是独立的，我们将所有观察结果相乘。对于特定的$i$，一种聪明的方法是，回想$D_i$只能是1或0：

$$\left( p(t_1 \; | \; \hat{\beta}, \hat{\alpha} )\right)^{D_i} \left( 1 - p(t_1 \; | \; \hat{\beta}, \hat{\alpha} ) \right)^{(1-D_i)}$$


（这个数量可能会溢出，所以建议取上述的$\log$：

$$ D_i\log( p(t_1 \; | \; \hat{\beta}, \hat{\alpha} ) ) + (1-D_i)\log( 1- p(t_1 \; | \; \hat{\beta}, \hat{\alpha} ) ) $$

并对项求和而不是相乘。确保对比的两个模型都使用这个转换。）

In [3]:
print p_hat
_product = p_hat ** (D) * (1 - p_hat) ** (1 - D)
prob_given_model_1 = _product.prod()
print "probability of observations, model 1: %.10f" % prob_given_model_1

NameError: name 'p_hat' is not defined

我们正在与之比较的第二个模型是$\beta=0$，也就是说，概率和温度之间没有关系。我们执行与上面相同的计算。请注意，在下面的PyMC代码中，`beta=0`。

In [4]:
beta = 0
alpha = pm.Normal("alpha", 0, 0.001, value=0)


@pm.deterministic
def p(temp=temperature, alpha=alpha, beta=beta):
    return 1.0 / (1. + np.exp(beta * temperature + alpha))


observed = pm.Bernoulli("bernoulli_obs", p,
                        value=D, observed=True)

model = pm.Model({"observed": observed, "beta": beta, "alpha": alpha})

# mysterious code to be explained in Chapter 3
map_ = pm.MAP(model)
map_.fit()
mcmc = pm.MCMC(model)
mcmc.sample(260000, 220000, 3)
######

alpha_samples_model_2 = mcmc.trace("alpha")[:, None]
alpha_hat = alpha_samples_model_2[0]  # use the first 'model'
beta_hat = 0

p_hat = logistic(temperature, beta_hat, alpha_hat)
print "estimates of probability at observed temperature, defects: "
print np.array(zip(p_hat, temperature, D))[:3, :]
print
print "Notice the probability is constant for all temperatures?"

NameError: name 'mc' is not defined

In [5]:
# compute the probability of observations given the model_2

_product = p_hat ** (D) * (1 - p_hat) ** (1 - D)
prob_given_model_2 = _product.prod()
print "probability of observations, model 2: %.10f" % prob_given_model_2

NameError: name 'p_hat' is not defined

In [7]:
print "Bayes factor = %.3f" % (prob_given_model_1 / prob_given_model_2)

NameError: name 'prob_given_model_1' is not defined

这个好吗？下面是一个可以用来比较计算得出的贝叶斯因子与对M1的置信度的图表。

 我们还没有完成。如果您记得，我们只选了一个模型，但请记得我们实际上有可能的模型分布（从后验分布中的顺序对（β，α）中）。因此，为了正确，我们需要对后验样本进行平均：

In [9]:
p = logistic(temperature[None, :], beta_samples, alpha_samples)
_product = p ** (D) * (1 - p) ** (1 - D)
prob_given_model_1 = _product.prod(axis=1).mean()
print "expected prob. of obs., given model 1: %.10f" % prob_given_model_1


p_model_2 = logistic(temperature[:, None],
                     np.zeros_like(temperature), alpha_samples_model_2)

_product = p_model_2 ** (D) * (1 - p_model_2) ** (1 - D)
prob_given_model_2 = _product.prod(axis=1).mean()
print "expected prob. of obs., given model 2: %.10f" % prob_given_model_2
print
print "Bayes factor: %.3f" % (prob_given_model_1 / prob_given_model_2)

NameError: name 'logistic' is not defined

看起来我们有一个相当好的模型，温度确实很重要。看看你，现在你是一个火箭科学家。